In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.append(str(PROJECT_ROOT))


from src.rag.vector_store import ComplaintVectorStore
from src.rag.retriever import ComplaintRetriever
from src.rag.generator import ComplaintGenerator
from src.rag.pipeline import ComplaintRAGPipeline
from src.rag.evaluate import evaluate


c:\Users\amare\Documents\KAIM\week-7\rag-complaint-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize vector store
vector_store = ComplaintVectorStore(
    parquet_path="../data/complaint_embeddings.parquet"
)

# Build FAISS index
vector_store.build_faiss_index()



Building FAISS index: 100%|██████████| 28/28 [00:12<00:00,  2.18it/s]


<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023FA3861950> >

In [3]:
# Retriever using the vector store
retriever = ComplaintRetriever(
    vector_store=vector_store
)

# Generator
generator = ComplaintGenerator()


Device set to use cpu


✓ ComplaintGenerator loaded: google/flan-t5-small


In [4]:
# Connect retriever + generator
rag_pipeline = ComplaintRAGPipeline(
    retriever=retriever,
    generator=generator
)


In [5]:
question = "What issues do customers report with credit card billing?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: What issues do customers report with credit card billing?
Answer:
 I have never had to complain about the service i was getting from a credit card company

Sample Source Metadata: {'chunk_index': 0, 'company': 'CAPITAL ONE FINANCIAL CORPORATION', 'complaint_id': '2717332', 'date_received': '2017-11-01', 'issue': 'Trouble using your card', 'product': 'Credit card or prepaid card', 'product_category': 'Credit Card', 'state': 'CA', 'sub_issue': "Can't use card to make purchases", 'total_chunks': 16}


In [6]:
question = "Are there common problems with money transfer delays?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: Are there common problems with money transfer delays?
Answer:
 I am worried that this delay would not only cost me money, but will also violate banking standards on the timely availability of funds.

Sample Source Metadata: {'chunk_index': 2, 'company': 'Coinbase, Inc.', 'complaint_id': '2601401', 'date_received': '2017-08-11', 'issue': 'Money was not available when promised', 'product': 'Money transfer, virtual currency, or money service', 'product_category': 'Money Transfer', 'state': 'NV', 'sub_issue': 'nan', 'total_chunks': 3}


In [7]:
question = "What complaints are frequent for personal loans?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: What complaints are frequent for personal loans?
Answer:
 constantly being harassed by employees about a loan that is not the ones issueing the loan, absolutely ridiculous

Sample Source Metadata: {'chunk_index': 0, 'company': 'MARINER FINANCE, LLC', 'complaint_id': '6456989', 'date_received': '2023-01-20', 'issue': "Charged fees or interest you didn't expect", 'product': 'Payday loan, title loan, or personal loan', 'product_category': 'Personal Loan', 'state': 'TX', 'sub_issue': 'nan', 'total_chunks': 1}


In [8]:
question = "Are savings account fees a common concern?"

result = rag_pipeline.answer(question)

print("Question:", result["question"])
print("Answer:\n", result["answer"])
print("\nSample Source Metadata:", result["sources"][0]["metadata"])


Question: Are savings account fees a common concern?
Answer:
 no

Sample Source Metadata: {'chunk_index': 0, 'company': 'BANK OF AMERICA, NATIONAL ASSOCIATION', 'complaint_id': '5989439', 'date_received': '2022-09-16', 'issue': 'Problem caused by your funds being low', 'product': 'Checking or savings account', 'product_category': 'Savings Account', 'state': 'OR', 'sub_issue': 'Non-sufficient funds and associated fees', 'total_chunks': 6}


In [9]:
# Run all predefined evaluation questions
eval_df = evaluate(rag_pipeline)
eval_df


,Question,Generated Answer,Retrieved Source 1,Quality Score (1-5),Comments
0,What issues do customers report with credit ca...,I have never had to complain about the service...,as both a business owner and a head of househo...,,
1,Are there common problems with money transfer ...,I am worried that this delay would not only co...,o people like myself can make a decision to no...,,
2,What complaints are frequent for personal loans?,constantly being harassed by employees about a...,constantly being harassed by employees about a...,,
3,Do customers report unauthorized transactions?,No,unauthorized transactions and no customer serv...,,
4,Are savings account fees a common concern?,no,"like many people, i don't really use my saving...",,


In [10]:
def df_to_markdown(df):
    """Convert a DataFrame to a Markdown table string without external libraries."""
    # Header
    md = "| " + " | ".join(df.columns) + " |\n"
    md += "| " + " | ".join(["---"] * len(df.columns)) + " |\n"

    # Rows
    for _, row in df.iterrows():
        md += "| " + " | ".join(str(x) for x in row.values) + " |\n"
    return md

# Usage
print(df_to_markdown(eval_df))


| Question | Generated Answer | Retrieved Source 1 | Quality Score (1-5) | Comments |
| --- | --- | --- | --- | --- |
| What issues do customers report with credit card billing? | I have never had to complain about the service i was getting from a credit card company | as both a business owner and a head of household, i have maintained dozens of credit cards during the last 30 years, and i have never had to complain about the service i was getting from a credit card |  |  |
| Are there common problems with money transfer delays? | I am worried that this delay would not only cost me money, but will also violate banking standards on the timely availability of funds. | o people like myself can make a decision to not initiate the transfer in the first place. i was not looking to have my funds tied up for any amount of time, and these delays could be costing me real m |  |  |
| What complaints are frequent for personal loans? | constantly being harassed by employees about a loan that is not